In [1]:
# !pip install qiskit
# !pip install qiskit-aer
# !pip show qiskit-aer


In [2]:
from qiskit import *
import matplotlib.pyplot as plt # for plotting purpose
import numpy as np

from qiskit_aer import *
from qiskit import transpile  # instead of EXCUTE command ..it runs the job


In [3]:

my_list = [1,3,5,2,4,9,5,8,0,7,6]

In [4]:
def the_oracle(my_input):
  winner=7
  if my_input is winner:
    response = True
  else:
    response = False
  return response

In [5]:
for index, trial_number in enumerate(my_list):
  if the_oracle(trial_number) is True:
    print('wineer found at index %i'%index)
    print('%i calls to the Oracle used'%(index+1))
    break

wineer found at index 9
10 calls to the Oracle used


In [6]:
oracle = QuantumCircuit(2,name='oracle')  #define the oracle of 2 quibits [0,1] and name oracle
oracle.cz(0,1)  #change the sign of winner is done by cz gate
oracle.to_gate() #make oracle into its own gate (forming gate(ckt) of oracle)
oracle.draw()   #to draw ckt
# currently cz gate act as a oracle

q_0: ─■─
      │ 
q_1: ─■─

In [7]:
#to check wether the oracle is doing the right thing or not...we can ckeck this by preparing
# a superposition state of all my qubits by applying a hadmard gate on each one of them
# and this way we can query each one of the i/p state simultaneously to the oracle

# let call state s for superpostion
backend = Aer.get_backend('statevector_simulator')
                                                                                # query ==RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR

grover_circ = QuantumCircuit(2,2) #define grover ckt : 2 quibits and 2 classical registers
grover_circ.h([0,1])  #this is hadamard gate is used for superposition (prepares the superposition state s) it prepares all 4 superposition states [(00,01,10,11)]
grover_circ.append(oracle,[0,1]) # adding oracle on grover ckt so that we can query each one of states at same time
grover_circ.draw()  #to draw grover ckt


┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [8]:
new_circ = transpile(grover_circ, backend)  # to execute this job on simulator
job = backend.run(new_circ)
result = job.result() # get back the result

In [9]:
sv = result.get_statevector() # to get state vector
np.around(sv,2) # in result we get the one state flipped (11)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [10]:
# reflection operator is used for amplitude amplification
# anplitude modification is to
  # --> amplify the probabilities of winning state w ans
  # --> reduce  the probabilities of non winning state
reflection = QuantumCircuit(2,name='reflection')
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()


Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [11]:
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [12]:
backend = Aer.get_backend('qasm_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,[0,1])
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])

In [13]:
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [14]:
new_circ = transpile(grover_circ, backend)
job = backend.run(new_circ,shots=1)
result = job.result()
result.get_counts()

{'11': 1}